In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
a = input()
print(a)

In [4]:
from os import path
import pandas as pd
import sys
import json
import torch
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import os


import util as tu
import model as tm
from setup import *


In [5]:

class Args:
    test_database_ids = [13]
    random_seed = 114514
    batch_size = 1000
    accelerator = 'cpu'
    progress_bar = True
    lr = 0.001
    epochs = 200
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cpu:0'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()
configs = vars(args)
print(args)

In [6]:
with open(path.join(ROOT_DIR, 'data', 'workload1', 'statistics.json')) as file:
    feature_statistics = json.load(file)
encoder = tu.Encoder(feature_statistics)

{'Index Only Scan': 0, 'Index Scan': 1, 'Seq Scan': 2, 'Hash Join': 3, 'Bitmap Index Scan': 4, 'Result': 5, 'Merge Join': 6, 'Limit': 7, 'Aggregate': 8, 'Sort': 9, 'Hash': 10, 'Bitmap Heap Scan': 11, 'Nested Loop': 12, 'Materialize': 13, 'BitmapOr': 14, 'Memoize': 15}
{'Actual Total Time': RobustScaler(), 'Plan Rows': RobustScaler(), 'Total Cost': RobustScaler()}


In [7]:
files_name = []
for wk_item in workloads:
    files_name.append(path.join(ROOT_DIR, 'data', 'workload1', wk_item + '_filted.json'))

dict_list = tu.Encoder.format_workload(files_name)
alldataset = tu.PlanTreeDataSet(dict_list, encoder)


torch.Size([200000, 1, 360]) torch.Size([200000, 20, 20]) torch.Size([200000, 1]) torch.Size([200000, 1])


In [9]:
train_dataloader, val_dataloader, test_dataloader = tu.get_dataloader(alldataset, args)

5 4 4


In [10]:
model = tm.SeqFormer(args, input_dim=18, hidden_dim=192, output_dim=1)
logger = pl.loggers.WandbLogger(project="DACE")
logger.log_hyperparams(args)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=os.path.join(ROOT_DIR, "checkpoints"),
    filename="DACE",
)
trainer =pl.Trainer(
    accelerator=args.accelerator,
    enable_progress_bar=args.progress_bar,
    enable_model_summary=args.progress_bar,
    max_epochs=args.max_epoch,
    logger=logger,
    callbacks=[checkpoint_callback],
)

{}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanji. Use `wandb login --relogin` to force relogin


KeyError: 'accelerator'

In [ ]:
# trainer = PLTrainer(accelerator="cpu", max_epochs=50, logger=wandb_logger)
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
# test model
result = trainer.test(model, dataloaders=test_dataloader)